In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)
from datetime import datetime
import numpy as np
import timeit
import json

In [2]:
start_time = timeit.default_timer()


df_2019_sep = pd.read_json('json_datasets/Kickstarter_2019-09-12T03_20_06_215Z.json', lines=True)


elapsed = timeit.default_timer() - start_time
print(f'time: {elapsed}')

time: 107.045443821


In [3]:
# Save the creation datetime for calculating age of project later
df_2019_sep_collected_on = datetime(2019, 9, 12)

In [4]:
# clean up the format and make a deep copy
start_time = timeit.default_timer()


df_2019_sep = df_2019_sep['data'].apply(pd.Series)
df_2019_sep_copy = df_2019_sep.copy(deep=True)
print(f'Total_rows: {len(df_2019_sep)}')


elapsed = timeit.default_timer() - start_time
print(f'time: {elapsed}')

df_2019_sep.head(3)

Total_rows: 213657
time: 258.693635383


,id,photo,name,blurb,goal,pledged,state,slug,disable_communication,country,currency,currency_symbol,currency_trailing_code,deadline,state_changed_at,created_at,launched_at,staff_pick,is_starrable,backers_count,static_usd_rate,usd_pledged,converted_pledged_amount,fx_rate,current_currency,usd_type,creator,location,category,profile,spotlight,urls,source_url,friends,is_starred,is_backing,permissions
0,1257739938,{'key': 'assets/024/662/100/c07f6483eca0a67c3b...,COASTIX: Our Coaster. Your Pix. Infinite Possi...,Bring your favorite social media photos and ar...,5000.0,5545.29,successful,coastix-our-coaster-your-pix-infinite-possibil...,False,CA,CAD,$,True,1557403845,1557403845,1519358040,1554811845,False,False,77,0.74748,4144.9916501601,4112,0.760917,USD,domestic,"{'id': 413554068, 'name': 'Austin K.', 'slug':...","{'id': 8775, 'name': 'Calgary', 'slug': 'calga...","{'id': 28, 'name': 'Product Design', 'slug': '...","{'id': 3317372, 'project_id': 3317372, 'state'...",True,{'web': {'project': 'https://www.kickstarter.c...,https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN
1,160619929,{'key': 'assets/012/305/450/5a814a89611f186edb...,Photo Caffe,Photo Cafe is a hybrid of a photography studio...,500000.0,0.00,failed,photo-caffe,False,US,USD,$,True,1452631347,1452631348,1447287592,1447447347,False,False,0,1.00000,0.0,0,1.000000,USD,domestic,"{'id': 1633535727, 'name': 'David Seybold', 'i...","{'id': 2484861, 'name': 'Roseville', 'slug': '...","{'id': 276, 'name': 'Fine Art', 'slug': 'photo...","{'id': 2224280, 'project_id': 2224280, 'state'...",False,{'web': {'project': 'https://www.kickstarter.c...,https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN
2,452313895,{'key': 'assets/012/294/084/27afadb270e23d7ffe...,Artemis: a collaboration captured on medium fo...,"A collaboration between Claire Pettibone, Trum...",2500.0,10.00,failed,artemis-a-collaboration-captured-on-medium-format,False,US,USD,$,True,1447269352,1447269352,1446303312,1446750952,False,False,1,1.00000,10.0,10,1.000000,USD,domestic,"{'id': 1186970994, 'name': 'Travis Kaenel', 'i...","{'id': 2442047, 'name': 'Los Angeles', 'slug':...","{'id': 276, 'name': 'Fine Art', 'slug': 'photo...","{'id': 2202735, 'project_id': 2202735, 'state'...",False,{'web': {'project': 'https://www.kickstarter.c...,https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN


In [5]:
# Check if data has same features
df_2019_sep_cols = df_2019_sep.columns
columns_original  = ['id', 'photo', 'name', 'blurb', 'goal', 'pledged', 'state', 'slug', 'disable_communication', 'country', 'country_displayable_name', 'currency', 'currency_symbol', 'currency_trailing_code', 'deadline', 'state_changed_at', 'created_at', 'launched_at', 'staff_pick', 'is_starrable', 'backers_count', 'static_usd_rate', 'usd_pledged', 'converted_pledged_amount', 'fx_rate', 'current_currency', 'usd_type', 'creator', 'location', 'category', 'profile', 'spotlight', 'urls', 'source_url', 'friends', 'is_starred', 'is_backing', 'permissions']
diff = list(set(df_2019_sep_cols) - set(columns_original))
diff2 = list(set(columns_original) - set(df_2019_sep_cols))
print(diff)
print(diff2)

[]
['country_displayable_name']


In [6]:
# inspect canceled and suspended states
df_2019_sep.state.value_counts()

successful    121978
failed         75634
canceled        8725
live            6679
suspended        641
Name: state, dtype: int64

In [7]:
# drop canceled and suspended states
df_2019_sep.drop((df_2019_sep[((df_2019_sep.state == 'suspended') | 
                                         (df_2019_sep.state == 'canceled'))].index), inplace=True)

# check for duplicates
print(f"Rows with duplicate project_id: {df_2019_sep.duplicated(['id']).sum()}")

# drop duplicates
df_2019_sep.drop_duplicates(['id'], inplace=True)
print(f'Total_rows after dropping duplicates and suspended/canceled states: {len(df_2019_sep)}')


Rows with duplicate project_id: 27304
Total_rows after dropping duplicates and suspended/canceled states: 176987


In [8]:
# # get links to project and creator pages
# start_time = timeit.default_timer()


# df_2019_sep['project_link'] = 'no_val'
# project_link_loc = df_2019_sep.columns.get_loc('project_link')
# for n in range(len(df_2019_sep)):
#     project_link = df_2019_sep.iloc[n].urls['web']['project']
#     df_2019_sep.iat[n, project_link_loc] = project_link
    
# df_2019_sep['creator_link'] = 'no_val'
# creator_link_loc = df_2019_sep.columns.get_loc('creator_link')
# for n in range(len(df_2019_sep)):
#     creator_link = df_2019_sep.iloc[n].creator['urls']['web']['user']
#     df_2019_sep.iat[n, creator_link_loc] = creator_link  
    
    
# elapsed = timeit.default_timer() - start_time
# print(f'time: {elapsed}')

In [9]:
# # get main_category id
# start_time = timeit.default_timer()


# categories_map = {1:'Art',3:'Comics', 17:'Theater', 15:'Photography', 
#                 7:'Design', 6: 'Dance', 11:'Film & Video', 12:'Games', 16:'Technology', 
#                 13:'Journalism', 18:'Publishing', 9:'Fashion', 10:'Food', 14:'Music', 26:'Crafts'}
# categories_df = pd.DataFrame(categories_map, index=[0])


# df_2019_sep['main_category'] = 'no_value'
# cat_loc = df_2019_sep.columns.get_loc('main_category')


# for n in (range(len(df_2019_sep))):
#     if 'parent_id' not in df_2019_sep.iloc[n]['category'].keys():
#         cat_id = df_2019_sep.iloc[n]['category']['id']
#         df_2019_sep.iat[n, cat_loc] = cat_id
#     elif pd.isnull(df_2019_sep.iloc[n]['category']['parent_id']):
#         cat_id = df_2019_sep.iloc[n]['category']['id']
#         df_2019_sep.iat[n, cat_loc] = cat_id
#     else:
#         par_id = df_2019_sep.iloc[n]['category']['parent_id']
#         df_2019_sep.iat[n, cat_loc] = par_id
        
# df_2019_sep['main_category'] = df_2019_sep['main_category'].apply(lambda x: categories_df[x])


# elapsed = timeit.default_timer() - start_time
# print(f'time: {elapsed}')

# df_2019_sep.head(1)

In [10]:
# keep these features
df_main_feats = ['id', 'name', 'blurb', 'goal', 'pledged', 'state', 'country',
'currency', 'deadline', 'launched_at', 'staff_pick', 'main_category',
'backers_count', 'static_usd_rate', 'usd_pledged' ]
# Drop these 
df_drop_feats = list(set(df_2019_sep_cols) - set(df_main_feats))
print(f'Columns removed: {df_drop_feats}')

# drop features
df_2019_sep.drop(df_drop_feats, axis=1, inplace=True)

Columns removed: ['currency_symbol', 'currency_trailing_code', 'disable_communication', 'slug', 'is_starred', 'photo', 'urls', 'created_at', 'current_currency', 'usd_type', 'location', 'is_starrable', 'fx_rate', 'category', 'permissions', 'spotlight', 'is_backing', 'friends', 'state_changed_at', 'profile', 'creator', 'source_url', 'converted_pledged_amount']


In [11]:
# convert unix dates to datetime objects
df_2019_sep.deadline = df_2019_sep.deadline.apply(lambda x: (datetime.fromtimestamp(x)))
df_2019_sep.launched_at = df_2019_sep.launched_at.apply(lambda x: (datetime.fromtimestamp(x)))

In [12]:
# rename columns
df_2019_sep.rename(columns={'id':'project_id'}, inplace=True)
df_2019_sep.rename(columns={'name':'project_name'}, inplace=True)
df_2019_sep.rename(columns={'blurb':'project_description'}, inplace=True)
df_2019_sep.rename(columns={'state':'outcome_state'}, inplace=True)


In [13]:
# add project duration feature
df_2019_sep['duration'] = df_2019_sep.deadline - df_2019_sep.launched_at
df_2019_sep['days_from_launch'] =(df_2019_sep_collected_on - df_2019_sep.launched_at).apply(lambda x: x.days)

# add feature 'in_mid_duration_range' indicating whether project is about halfway through
df_2019_sep['duration_mid_point'] = df_2019_sep.duration.apply(lambda x: x.days) / 2
df_2019_sep['lower_bound_mid_point_range'] = df_2019_sep.duration_mid_point.apply(lambda x: x - 5)
df_2019_sep['upper_bound_mid_point_range'] = df_2019_sep.duration_mid_point.apply(lambda x: x + 5)
df_2019_sep['in_mid_duration_range'] = df_2019_sep.days_from_launch.between(df_2019_sep.lower_bound_mid_point_range, df_2019_sep.upper_bound_mid_point_range)

df_2019_sep.drop(['duration_mid_point', 'lower_bound_mid_point_range', 'upper_bound_mid_point_range'], axis=1, inplace=True)


In [14]:
# df_2019_sep['duration'][0].quantile(.1)
df_2019_sep['duration_10_percent'] = df_2019_sep.duration.apply(lambda x: x.days / 10 ) 


In [15]:
# add feature 'in_mid_duration_range' indicating whether project is about halfway through
df_2019_sep['lower_bound'] = df_2019_sep.duration_10_percent.apply(lambda x: x if x < 5 else x - 5)
df_2019_sep['upper_bound'] = df_2019_sep.duration_10_percent.apply(lambda x: x + 5)
df_2019_sep['in_range'] = df_2019_sep.days_from_launch.between(df_2019_sep.lower_bound, df_2019_sep.upper_bound)

# df_2019_sep.drop(['duration_mid_point', 'lower_bound_mid_point_range', 'upper_bound_mid_point_range'], axis=1, inplace=True)

In [16]:
# calculate usd_goal
df_2019_sep['usd_goal'] = df_2019_sep.goal * df_2019_sep.static_usd_rate


In [17]:
# Replace goal and rearrange
col_list = df_2019_sep.columns.to_list()
print(col_list)
col_list_rearrange = ['project_id', 'project_name', 'project_description', 'outcome_state', 'in_range',
                      'duration_10_percent', 'days_from_launch', 
                      'usd_goal', 'usd_pledged', 'currency', 'launched_at', 'deadline', 'staff_pick', 'country', 
                      'duration' , 'backers_count']
df_2019_sep = df_2019_sep[col_list_rearrange]
df_2019_sep.head(1)
                      

['project_id', 'project_name', 'project_description', 'goal', 'pledged', 'outcome_state', 'country', 'currency', 'deadline', 'launched_at', 'staff_pick', 'backers_count', 'static_usd_rate', 'usd_pledged', 'duration', 'days_from_launch', 'in_mid_duration_range', 'duration_10_percent', 'lower_bound', 'upper_bound', 'in_range', 'usd_goal']


,project_id,project_name,project_description,outcome_state,in_range,duration_10_percent,days_from_launch,usd_goal,usd_pledged,currency,launched_at,deadline,staff_pick,country,duration,backers_count
0,1257739938,COASTIX: Our Coaster. Your Pix. Infinite Possi...,Bring your favorite social media photos and ar...,successful,False,3.0,155,3737.39845,4144.9916501601,CAD,2019-04-09 08:10:45,2019-05-09 08:10:45,False,CA,30 days,77


In [18]:
df_2019_sep_copy = df_2019_sep.copy(deep=True)
# convert datetime columns before exporting
# df_2019_sep.duration = df_2019_sep.duration.apply(lambda x: x.days) 
df_2019_sep.launched_at = df_2019_sep.launched_at.apply(lambda x: x.strftime('%m-%d-%Y'))
df_2019_sep.deadline = df_2019_sep.deadline.apply(lambda x: x.strftime('%m-%d-%Y'))

In [19]:
# export cleaned dataframe to csv
df_2019_sep.to_csv('clean_datasets_10/main_df_2019_sep.csv', index=False)